### kyobo test


In [1]:
from concurrent.futures import ProcessPoolExecutor
from scraping import scrap_book_info
import asyncio
import pandas as pd

# 없는 경우 9791188174287

df = pd.read_parquet("../../App/db/data/book_info.parquet")

isbns = df["ISBN"].tolist()[:20]

chunk = 100
answer = []

async def scraping(task) -> map : 
     return await asyncio.run(task)

for i in range(0, len(isbns), chunk):
        chunk_isbns = isbns[i : i + chunk]
        tasks = [scrap_book_info(isbn) for isbn in chunk_isbns]
        with ProcessPoolExecutor() as executor:
            results: map = executor.map(scraping,tasks)
            answer.append(results)

        if i % 300 == 0 and i != 0:
            print(f"{round(i/len(isbns),3)*100}")


In [ ]:
# import requests
# from bs4 import BeautifulSoup
# # while True :
# ISBN = df.ISBN.sample(1)
# ISBN = ISBN.tolist()[0]
# libUrl = f"http://www.kyobobook.co.kr/product/detailViewKor.laf?ejkGb=KOR&mallGb=KOR&barcode={ISBN}"
# soup = BeautifulSoup(requests.get(libUrl,timeout=200).content,'html.parser')
# print(ISBN)
# soup.find(class_="prod_title").string


### lib api test


In [ ]:
from scraping import update_book_data
import asyncio
from datetime import date

lib_codes = [
    111003,
    111004,
    111005,
    111006,
    111007,
    111008,
    111009,
    111010,
    111022,
    111011,
    111012,
    111013,
    111014,
    111016,
    111030,
    111015,
    111018,
    111019,
    111020,
    111021,
]

chunk = 10
answer = []
for i in range(0, len(lib_codes), chunk):
    chunk_lib_codes = lib_codes[i : i + chunk]
    tasks = [update_book_data(i, start_date=date(2020, 1, 1)) for i in chunk_lib_codes]
    results = await asyncio.gather(*tasks)
    answer.extend(results)


In [2]:
import aiofiles
import aiohttp
from bs4 import BeautifulSoup


async def _download_book_cover(soup: BeautifulSoup, dir: str) -> None:
    async with aiohttp.ClientSession() as session:
        url = soup.find(name="meta", property="og:image")["content"]
        async with session.get(url) as resp:
            if resp.status == 200:
                async with aiofiles.open(f"{dir}", "wb") as f:
                    await f.write(await resp.read())

ISBN = "9791158393717"
url = f"http://www.kyobobook.co.kr/product/detailViewKor.laf?ejkGb=KOR&mallGb=KOR&barcode={ISBN}"

async with aiohttp.ClientSession(trust_env=True) as session:
    async with session.get(url) as resp:
        html = await resp.read()

kyobo_soup = BeautifulSoup(html.decode("utf-8"), "html.parser")




In [11]:
kyobo_soup.find(name='meta',property='og:image')['content']

'https://contents.kyobobook.co.kr/sih/fit-in/458x0/pdt/9791158393717.jpg'